#### **Time Series Training Pipeline** <br>
This class provides a complete workflow to train, optimize, and evaluate machine learning models (LightGBM) for forecasting park occupancy rates (PopularTimesLivePercent) across multiple parks. <br>
Each park is modeled independently, enabling park-specific performance tracking and model tuning.

In [1]:
import pandas as pd

# Load data into pandas DataFrame 
imputed_fact_live_times = spark.read.table("imputed_fact_live_times").toPandas()

StatementMeta(, a9ceec60-ae88-4d5b-9a36-1e4940d1ad38, 3, Finished, Available, Finished)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import optuna
import random
import os
import json
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import ttest_rel

def set_global_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_global_seed(42)

class ModularMultiParkLightGBM:
    def __init__(self, df, target_col='PopularTimesLivePercent', park_col='ParkName'):
        self.df = df.copy()
        self.target_col = target_col
        self.park_col = park_col
        
        # Store different model versions
        self.baseline_models = {}
        self.optimized_models = {}
        self.final_models = {}
        
        # Store results
        self.baseline_results = {}
        self.optimized_results = {}
        self.final_results = {}
        self.cv_results = {}
        self.statistical_tests = {}

        self.exog_features = ['Temp', 'IsRaining', 'IsWeekend', 'HasEvent', 'IsSnowing', 'IsHoliday']
        print("Modular Pipeline initialized.")
        
        # Print data info
        print(f"Dataset range: {df.index.min()} to {df.index.max()}")
        print(f"Total parks: {df[park_col].nunique()}")
        print(f"Total observations: {len(df)}")

    def create_features(self):
        """Simplified feature creation for clean dataframe"""
        df = self.df.copy()
        
        # Simple time features
        df['HourOp'] = (df['Hour'] - 6) % 17
        df['HourSin'] = np.sin(2 * np.pi * df['HourOp'] / 17)
        df['HourCos'] = np.cos(2 * np.pi * df['HourOp'] / 17)

        # Lag features
        for lag in [1, 2, 3, 17]:
            df[f'Lag_{lag}'] = df.groupby(self.park_col)[self.target_col].shift(lag)

        # Day-of-Week–aware 6AM lag imputation
        for park in df[self.park_col].unique():
            park_mask = df[self.park_col] == park
            park_df = df[park_mask]

            # Compute average occupancy at 6AM per DayOfWeek
            avg_6am_by_day = (
                park_df[(park_df['Hour'] == 6) & park_df[self.target_col].notna()]
                .groupby('DayOfWeek')[self.target_col]
                .mean()
            )

            # Identify 6AM rows for this park
            six_am_mask = (df['Hour'] == 6) & park_mask

            # Inject DOW-aware lag values
            dow_values = df.loc[six_am_mask, 'DayOfWeek']
            for lag in [1, 2, 3, 17]:
                df.loc[six_am_mask, f'Lag_{lag}'] = dow_values.map(avg_6am_by_day)

        self.df = df[df[self.target_col].notna()]  
        print(f"After feature creation and dropna: {len(self.df)} observations")

    def train_baseline_models(self, test_size=0.2):
        """Train baseline models with default parameters"""
        print(" Step 1: Training baseline models...")
        parks = self.df[self.park_col].unique()

        all_features = self.exog_features + ['HourOp', 'HourSin', 'HourCos', 'DayOfWeek'] + [f'Lag_{lag}' for lag in [1, 2, 3, 17]]

        for i, park in enumerate(parks):
            park_data = self.df[self.df[self.park_col] == park].sort_index()
            
            if len(park_data) < 100:
                print(f"Skipping {park} - insufficient data ({len(park_data)} observations)")
                continue
            
            split = int(len(park_data) * (1 - test_size))
            train = park_data.iloc[:split]
            test = park_data.iloc[split:]
            
            # Print test period info for first park
            if i == 0:
                print(f"  Example test period: {test.index.min()} to {test.index.max()}")

            X_train, y_train = train[all_features], train[self.target_col]
            X_test, y_test = test[all_features], test[self.target_col]

            # Baseline model
            baseline_model = lgb.LGBMRegressor(
                learning_rate=0.1,
                num_leaves=31,
                feature_fraction=0.9,
                bagging_fraction=0.8,
                bagging_freq=5,
                reg_alpha=0.1,
                reg_lambda=0.1,
                verbosity=-1,
                random_state=42
            )

            baseline_model.fit(X_train, y_train)
            preds = baseline_model.predict(X_test)

            self.baseline_models[park] = baseline_model
            self.baseline_results[park] = {
                'test_dates': test.index,
                'actual': y_test,
                'predicted': preds,
                'metrics': {
                    'MAE': mean_absolute_error(y_test, preds),
                    'RMSE': np.sqrt(mean_squared_error(y_test, preds)),
                    'R2': r2_score(y_test, preds)
                }
            }
            
            print(f"  {i+1:2d}. {park[:30]:<30} | Baseline R²: {self.baseline_results[park]['metrics']['R2']:.3f}")

    def optimize_models(self, test_size=0.2, n_trials=20):
        """Optimize models using Optuna"""
        print(f"\n Step 2: Optimizing models ({n_trials} trials each)...")

        all_features = self.exog_features + ['HourOp', 'HourSin', 'HourCos', 'DayOfWeek'] + [f'Lag_{lag}' for lag in [1, 2, 3, 17]]
        for i, park in enumerate(self.baseline_models.keys()):
            print(f"  Optimizing {park[:30]:<30}...", end="")
            
            park_data = self.df[self.df[self.park_col] == park].sort_index()
            split = int(len(park_data) * (1 - test_size))
            train = park_data.iloc[:split]
            test = park_data.iloc[split:]

            # Optimization with validation split
            val_split = int(len(train) * 0.8)
            train_opt = train.iloc[:val_split]
            val_opt = train.iloc[val_split:]

            def objective(trial):
                params = {
                    'objective': 'regression',
                    'metric': 'rmse',
                    'verbosity': -1,
                    'learning_rate': trial.suggest_float("learning_rate", 0.05, 0.3),
                    'num_leaves': trial.suggest_int("num_leaves", 20, 150),
                    'feature_fraction': trial.suggest_float("feature_fraction", 0.6, 1.0),
                    'bagging_fraction': trial.suggest_float("bagging_fraction", 0.6, 1.0),
                    'bagging_freq': trial.suggest_int("bagging_freq", 1, 7),
                    'min_child_samples': trial.suggest_int("min_child_samples", 5, 100),
                    'reg_alpha': trial.suggest_float("reg_alpha", 0, 10),
                    'reg_lambda': trial.suggest_float("reg_lambda", 0, 10)
                }

                model = lgb.LGBMRegressor(**params)
                model.fit(train_opt[all_features], train_opt[self.target_col])
                preds = model.predict(val_opt[all_features])
                return np.sqrt(mean_squared_error(val_opt[self.target_col], preds))

            #study = optuna.create_study(direction="minimize")
            study = optuna.create_study(direction="minimize",sampler=optuna.samplers.TPESampler(seed=42))
            study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

            # Train optimized model
            #optimized_model = lgb.LGBMRegressor(**study.best_params)
            optimized_model = lgb.LGBMRegressor(**study.best_params, random_state=42)
            optimized_model.fit(train[all_features], train[self.target_col])
            preds = optimized_model.predict(test[all_features])

            self.optimized_models[park] = optimized_model
            self.optimized_results[park] = {
                'test_dates': test.index,
                'actual': test[self.target_col],
                'predicted': preds,
                'metrics': {
                    'MAE': mean_absolute_error(test[self.target_col], preds),
                    'RMSE': np.sqrt(mean_squared_error(test[self.target_col], preds)),
                    'R2': r2_score(test[self.target_col], preds)
                }
            }
            
            print(f" Optimized R²: {self.optimized_results[park]['metrics']['R2']:.3f}")

    def compare_and_select_best(self):
        """Statistical comparison and model selection"""
        print("\n Step 3: Comparing baseline vs optimized models...")
        
        improvement_count = 0
        
        for park in self.baseline_results.keys():
            baseline_r2 = self.baseline_results[park]['metrics']['R2']
            optimized_r2 = self.optimized_results[park]['metrics']['R2']
            baseline_mae = self.baseline_results[park]['metrics']['MAE']
            optimized_mae = self.optimized_results[park]['metrics']['MAE']
            
            # Statistical test
            baseline_residuals = self.baseline_results[park]['actual'] - self.baseline_results[park]['predicted']
            optimized_residuals = self.optimized_results[park]['actual'] - self.optimized_results[park]['predicted']
            
            baseline_abs_errors = np.abs(baseline_residuals)
            optimized_abs_errors = np.abs(optimized_residuals)
            
            t_stat, p_value = ttest_rel(baseline_abs_errors, optimized_abs_errors)
            
            # Decision criteria
            r2_improvement = optimized_r2 - baseline_r2
            mae_improvement = baseline_mae - optimized_mae
            
            significant_improvement = (p_value < 0.05) and (mae_improvement > 0)
            substantial_improvement = r2_improvement > 0.02
            
            if significant_improvement or substantial_improvement:
                self.final_models[park] = self.optimized_models[park]
                self.final_results[park] = self.optimized_results[park]
                choice = "Optimized"
                improvement_count += 1
            else:
                self.final_models[park] = self.baseline_models[park]
                self.final_results[park] = self.baseline_results[park]
                choice = "Baseline"
            
            self.statistical_tests[park] = {
                'baseline_r2': baseline_r2,
                'optimized_r2': optimized_r2,
                'r2_improvement': r2_improvement,
                'mae_improvement': mae_improvement,
                'p_value': p_value,
                'significant': significant_improvement,
                'choice': choice
            }
            
            print(f"  {park[:25]:<25} | Baseline: {baseline_r2:.3f} | Optimized: {optimized_r2:.3f} | Choice: {choice}")
        
        print(f"\n Summary: {improvement_count}/{len(self.baseline_results)} parks improved with optimization")

    def cross_validate_final_models(self, n_splits=5):
        """Cross-validate final models"""
        print(f"\n Step 4: Cross-validating final models ({n_splits} folds)...")
        
        all_features = self.exog_features + ['HourOp', 'HourSin', 'HourCos', 'DayOfWeek'] + [f'Lag_{lag}' for lag in [1, 2, 3, 17]]
        for park in self.final_models.keys():
            park_data = self.df[self.df[self.park_col] == park].sort_index()
            
            cv_scores = []
            n = len(park_data)
            
            for i in range(n_splits):
                train_end = int(n * (0.5 + 0.1 * i))
                val_start = train_end
                val_end = min(train_end + int(n * 0.15), n)
                
                if val_end <= val_start:
                    continue
                
                train_cv = park_data.iloc[:train_end]
                val_cv = park_data.iloc[val_start:val_end]
                
                X_train_cv = train_cv[all_features]
                y_train_cv = train_cv[self.target_col]
                X_val_cv = val_cv[all_features]
                y_val_cv = val_cv[self.target_col]
                
                if self.statistical_tests[park]['choice'] == 'Optimized':
                    model_cv = lgb.LGBMRegressor(**self.optimized_models[park].get_params())
                else:
                    model_cv = lgb.LGBMRegressor(**self.baseline_models[park].get_params())
                
                model_cv.fit(X_train_cv, y_train_cv)
                pred_cv = model_cv.predict(X_val_cv)
                
                mae_cv = mean_absolute_error(y_val_cv, pred_cv)
                cv_scores.append(mae_cv)
            
            self.cv_results[park] = {
                'cv_mae_scores': cv_scores,
                'cv_mae_mean': np.mean(cv_scores),
                'cv_mae_std': np.std(cv_scores),
                'cv_stability': np.std(cv_scores) / np.mean(cv_scores)
            }

    def plot_simple_predictions(self, parks_to_show="all", smooth_window=6):
        """Simple time series plots for all parks or specified number"""
        
        if parks_to_show == "all":
            parks_to_plot = list(self.final_results.keys())
            print(f"\n Plotting predictions for all {len(parks_to_plot)} parks...")
        else:
            # Get top performing parks
            parks_to_plot = sorted(self.final_results.keys(), 
                                  key=lambda x: self.final_results[x]['metrics']['R2'], 
                                  reverse=True)[:parks_to_show]
            print(f"\n Plotting predictions for top {parks_to_show} parks...")
        
        for park in parks_to_plot:
            result = self.final_results[park]
            actual = result['actual']
            predicted = pd.Series(result['predicted'], index=result['test_dates'])
            
            # Apply smoothing
            actual_smooth = actual.rolling(window=smooth_window, center=True).mean()
            predicted_smooth = predicted.rolling(window=smooth_window, center=True).mean()
            
            # Create the plot
            plt.figure(figsize=(16, 4))
            plt.plot(actual_smooth, label='Actual (Smoothed)', color='darkorange', linewidth=1.5)
            plt.plot(predicted_smooth, label='Predicted (Smoothed)', color='steelblue', linewidth=1.5)
            
            # Title with R² score and model choice
            r2 = result['metrics']['R2']
            choice = self.statistical_tests[park]['choice']
            plt.title(f"{park} Forecast - R²: {r2:.3f} (Model: {choice})")
            plt.ylabel('popularTimesLivePercent')
            plt.xlabel('Date')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

    def create_performance_summary(self):
        """Create a summary table of all parks"""
        print("\n PERFORMANCE SUMMARY TABLE:")
        print("-" * 80)
        print(f"{'Park Name':<30} {'R²':<8} {'MAE':<8} {'Model':<10} {'CV Stability':<12}")
        print("-" * 80)
        
        # Sort by R² performance
        sorted_parks = sorted(self.final_results.keys(), 
                             key=lambda x: self.final_results[x]['metrics']['R2'], 
                             reverse=True)
        
        for park in sorted_parks:
            r2 = self.final_results[park]['metrics']['R2']
            mae = self.final_results[park]['metrics']['MAE']
            choice = self.statistical_tests[park]['choice']
            
            if park in self.cv_results:
                cv_stability = self.cv_results[park]['cv_stability']
                cv_str = f"{cv_stability:.3f}"
            else:
                cv_str = "N/A"
            
            print(f"{park[:29]:<30} {r2:<8.3f} {mae:<8.1f} {choice:<10} {cv_str:<12}")

    def create_simple_visualizations(self, parks_to_show="all"):
        """Create simple, clean visualizations"""
        print("\n Creating Simple Visualizations...")
        print("="*40)
        
        # 1. Performance summary table
        self.create_performance_summary()
        
        # 2. Time series predictions for all parks
        self.plot_simple_predictions(parks_to_show=parks_to_show)
        
        print("\n Visualizations complete!")

    def run_full_pipeline(self, test_size=0.2, optimize_trials=20, cv_splits=5, create_visualizations=True, parks_to_show="all"):
        """Run the complete pipeline with simple visualizations"""
        print(" Running Full Modular Pipeline...")
        print("="*50)
        
        # Step 1: Feature engineering
        print(" Preparing features...")
        self.create_features()
        
        # Step 2: Baseline models
        self.train_baseline_models(test_size=test_size)
        
        # Step 3: Optimization
        self.optimize_models(test_size=test_size, n_trials=optimize_trials)
        
        # Step 4: Statistical comparison and selection
        self.compare_and_select_best()
        
        # Step 5: Cross-validation
        self.cross_validate_final_models(n_splits=cv_splits)
        
        # Step 6: Simple Visualizations
        if create_visualizations:
            self.create_simple_visualizations(parks_to_show=parks_to_show)
        
        print("\n Full pipeline complete!")
        return self.final_models, self.final_results, self.statistical_tests

    # Utility methods for individual use
    def get_park_summary(self, park_name):
        """Get detailed summary for a specific park"""
        if park_name not in self.final_results:
            print(f"Park '{park_name}' not found in results.")
            return None
        
        result = self.final_results[park_name]
        test = self.statistical_tests[park_name]
        
        print(f"\n DETAILED SUMMARY: {park_name}")
        print("-" * 50)
        print(f"Final Model Choice: {test['choice']}")
        print(f"Final R²: {result['metrics']['R2']:.3f}")
        print(f"Final MAE: {result['metrics']['MAE']:.2f}")
        print(f"Final RMSE: {result['metrics']['RMSE']:.2f}")
        print(f"Baseline R²: {test['baseline_r2']:.3f}")
        print(f"Optimized R²: {test['optimized_r2']:.3f}")
        print(f"R² Improvement: {test['r2_improvement']:.3f}")
        print(f"MAE Improvement: {test['mae_improvement']:.2f}")
        print(f"Statistical Significance (p-value): {test['p_value']:.4f}")
        
        if park_name in self.cv_results:
            cv = self.cv_results[park_name]
            print(f"CV MAE Mean: {cv['cv_mae_mean']:.2f}")
            print(f"CV MAE Std: {cv['cv_mae_std']:.2f}")
            print(f"CV Stability: {cv['cv_stability']:.3f}")
        
        return {
            'result': result,
            'statistical_test': test,
            'cv_result': self.cv_results.get(park_name, None)
        }

    def plot_single_park(self, park_name, smooth_window=6):
        """Plot prediction for a single park"""
        if park_name not in self.final_results:
            print(f"Park '{park_name}' not found in results.")
            return
        
        result = self.final_results[park_name]
        actual = result['actual']
        predicted = pd.Series(result['predicted'], index=result['test_dates'])
        
        # Apply smoothing
        actual_smooth = actual.rolling(window=smooth_window, center=True).mean()
        predicted_smooth = predicted.rolling(window=smooth_window, center=True).mean()
        
        # Create the plot
        plt.figure(figsize=(16, 4))
        plt.plot(actual_smooth, label='Actual (Smoothed)', color='darkorange', linewidth=1.5)
        plt.plot(predicted_smooth, label='Predicted (Smoothed)', color='steelblue', linewidth=1.5)
        
        # Title with detailed info
        r2 = result['metrics']['R2']
        mae = result['metrics']['MAE']
        choice = self.statistical_tests[park_name]['choice']
        plt.title(f"{park_name} Forecast - R²: {r2:.3f}, MAE: {mae:.1f} (Model: {choice})")
        plt.ylabel('popularTimesLivePercent')
        plt.xlabel('Date')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

StatementMeta(, 8e2299f2-db37-4db6-88d0-f86fc053c62e, 5, Finished, Available, Finished)

In [4]:
# Ensure Datetime is set as index
if not isinstance(imputed_fact_live_times.index, pd.DatetimeIndex):
    if 'Datetime' in imputed_fact_live_times.columns:
        imputed_fact_live_times = imputed_fact_live_times.set_index('Datetime').sort_index()
        print("Datetime column set as index.")
    else:
        raise ValueError("The dataframe does not have a 'Datetime' column to set as index.")
else:
    print("Datetime index already set.")

# Run your pipeline  
pipeline = ModularMultiParkLightGBM(imputed_fact_live_times)
models, results, stats = pipeline.run_full_pipeline(
    test_size=0.2,
    optimize_trials=15,
    cv_splits=3,
    parks_to_show="all"
)

StatementMeta(, 8e2299f2-db37-4db6-88d0-f86fc053c62e, 6, Submitted, Running, Running)

Datetime column set as index.
Modular Pipeline initialized.
Dataset range: 2024-05-01 06:00:00 to 2025-05-15 22:00:00
Total parks: 63
Total observations: 359762
 Running Full Modular Pipeline...
 Preparing features...
After feature creation and dropna: 359762 observations
 Step 1: Training baseline models...
  Example test period: 2025-03-02 18:00:00 to 2025-05-15 22:00:00


   1. Clinton Park                   | Baseline R²: 0.382


   2. Trillium Park                  | Baseline R²: 0.816


   3. Hillcrest Park                 | Baseline R²: 0.846


...

[I 2025-08-03 07:24:00,535] Trial 9 finished with value: 32.23823682121413 and parameters: {'learning_rate': 0.0513805292809006, 'num_leaves': 126, 'feature_fraction': 0.8827429375390468, 'bagging_fraction': 0.8916028672163949, 'bagging_freq': 6, 'min_child_samples': 12, 'reg_alpha': 3.5846572854427263, 'reg_lambda': 1.1586905952512971}. Best is trial 5 with value: 31.590032475232547.


[I 2025-08-03 07:24:07,908] Trial 10 finished with value: 32.44619267257613 and parameters: {'learning_rate': 0.2862841360575967, 'num_leaves': 20, 'feature_fraction': 0.747491509088439, 'bagging_fraction': 0.6071847502459278, 'bagging_freq': 1, 'min_child_samples': 71, 'reg_alpha': 9.597707459454197, 'reg_lambda': 6.765723797427796}. Best is trial 5 with value: 31.590032475232547.


[I 2025-08-03 07:24:19,293] Trial 11 finished with value: 31.599975727359045 and parameters: {'learning_rate': 0.10194395733023134, 'num_leaves': 82, 'feature_fraction': 0.692854583103252, 'bagging_fraction': 0.9783238879894853, 'bagging_freq': 1, 'min_child_samples': 99, 'reg_alpha': 3.9921404016484106, 'reg_lambda': 6.195339431335992}. Best is trial 5 with value: 31.590032475232547.


[I 2025-08-03 07:24:27,126] Trial 12 finished with value: 31.73121034710823 and parameters: {'learning_rate': 0.10000844821073998, 'num_leaves': 59, 'feature_fraction': 0.7067964291781478, 'bagging_fraction': 0.9872607155373294, 'bagging_freq': 2, 'min_child_samples': 55, 'reg_alpha': 4.321051437594004, 'reg_lambda': 6.471508029657881}. Best is trial 5 with value: 31.590032475232547.


[I 2025-08-03 07:24:34,757] Trial 13 finished with value: 31.907650847235857 and parameters: {'learning_rate': 0.22529064884844, 'num_leaves': 72, 'feature_fraction': 0.7185060051319042, 'bagging_fraction': 0.9984066126989742, 'bagging_freq': 2, 'min_child_samples': 77, 'reg_alpha': 2.860650014679747, 'reg_lambda': 6.243510430348395}. Best is trial 5 with value: 31.590032475232547.


### **Save Trained Model**

In [6]:
import os
import joblib

# Directory in Fabric Lakehouse to store the models
model_dir = "/lakehouse/default/Files/models/live_times"
os.makedirs(model_dir, exist_ok=True)

# Save each park's model
for park_name, model in pipeline.final_models.items():
    # Sanitize park name for filename
    filename = park_name.replace(" ", "_").replace("/", "_") + ".joblib"
    model_path = os.path.join(model_dir, filename)
    joblib.dump(model, model_path)
    
print(f"All models saved to: {model_dir}")


StatementMeta(, bb0d74a4-490a-43ff-9ed7-9863c7620f47, 8, Finished, Available, Finished)

All models saved to: /lakehouse/default/Files/models/live_times


### **Generate Summary of Model Evaluation**

In [18]:
# Generate summary of model evaluation
metric_summary_df = pd.DataFrame([
    {
        "park_name": park,
        "model_type": stats[park]['choice'],
        "R2": round(results[park]['metrics']['R2'] * 100, 2),
        "MAE": round(results[park]['metrics']['MAE'], 2),
        "RMSE": round(results[park]['metrics']['RMSE'], 2),
        "params_json": json.dumps(pipeline.final_models[park].get_params())
    }
    for park in results
])

spark_df = spark.createDataFrame(metric_summary_df)
spark_df.write.format("delta").mode("overwrite").saveAsTable("live_times_model_summary")


StatementMeta(, b4fcf156-1625-4ebf-86ec-269f818d6005, 20, Finished, Available, Finished)

In [20]:
# Save the parameters explicitly
params = model.get_params()
with open("model_params.json", "w") as f:
    json.dump(params, f)

StatementMeta(, b4fcf156-1625-4ebf-86ec-269f818d6005, 22, Finished, Available, Finished)